In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Transmute.AI/ChipNet/ChipNet-master

/content/drive/MyDrive/Transmute.AI/ChipNet/ChipNet-master


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Transmute.AI/ChipNet/ChipNet-master')

In [4]:
import argparse
import os

import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm as tqdm_notebook
from datasets import DataManager
from utils import *
import models
from models import get_model

In [5]:
seed_everything(43)

In [6]:
dataset = 'c100'
model = 'r164'
batch_size = 128
workers = 0
valid_size = 0.1
test_only = False
lr = 0.05
scheduler_type = [1,2]
decay = 0.001
epochs = 5
cuda_id = 0

In [7]:
data_object = DataManager(dataset, batch_size, workers, valid_size)
trainloader, valloader, testloader = data_object.prepare_data()
dataloaders = {
        'train': trainloader, 'val': valloader, "test": testloader
}

... Preparing data ...
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
using fixed split
45000 5000


In [8]:
model = get_model(model, 'full', data_object.num_classes, data_object.insize)

In [9]:
if os.path.exists('logs') == False:
    os.mkdir("logs")

if os.path.exists('checkpoints') == False:
    os.mkdir("checkpoints")

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=decay)

device = torch.device(f"cuda:{str(cuda_id)}")

model.to(device)


resnet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (1): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), 

In [10]:
def train(model, loss_fn, optimizer, scheduler=None):
    model.train()
    counter = 0
    tk1 = tqdm_notebook(dataloaders['train'], total=len(dataloaders['train']))
    running_loss = 0
    for x_var, y_var in tk1:
        counter +=1
        x_var = x_var.to(device=device)
        y_var = y_var.to(device=device)
        scores = model(x_var)

        loss = loss_fn(scores, y_var)
        running_loss+=loss.item()
        tk1.set_postfix(loss=running_loss/counter)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return running_loss/counter

In [11]:
def test(model, loss_fn, optimizer, phase, scheduler=None):
    model.eval()
    counter = 0
    tk1 = tqdm_notebook(dataloaders[phase], total=len(dataloaders[phase]))
    running_loss = 0
    running_acc = 0
    total = 0
    with torch.no_grad():
        for x_var, y_var in tk1:
            counter +=1
            x_var = x_var.to(device=device)
            y_var = y_var.to(device=device)
            scores = model(x_var)
            loss = loss_fn(scores, y_var)
            _, scores = torch.max(scores.data, 1)
            y_var = y_var.cpu().detach().numpy()
            scores = scores.cpu().detach().numpy()

            correct = (scores == y_var).sum().item()
            running_loss+=loss.item()
            running_acc+=correct
            total+=scores.shape[0]
            tk1.set_postfix(loss=running_loss/counter, acc=running_acc/total)
    return running_acc/total, running_loss/counter


In [14]:
best_acc = 0
num_epochs = epochs
train_losses = []
valid_losses = []
valid_accuracy = []
if test_only == False:
    for epoch in range(num_epochs):
        adjust_learning_rate(optimizer, epoch, scheduler_type, lr, epochs)
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        t_loss = train(model, criterion, optimizer)
        acc, v_loss = test(model, criterion, optimizer, "val")

        if acc>best_acc:
            print("**Saving model**")
            best_acc=acc
            torch.save({
                "epoch": epoch + 1,
                "state_dict" : model.state_dict(),
                "acc" : best_acc,
            }, f"checkpoints/r164_c10_pretrained.pth")

        train_losses.append(t_loss)
        valid_losses.append(v_loss)
        valid_accuracy.append(acc)
        df_data=np.array([train_losses, valid_losses, valid_accuracy]).T
        df = pd.DataFrame(df_data, columns = ['train_losses','valid_losses','valid_accuracy'])
        df.to_csv(f'logs/r164_c10_pretrained.csv')


Starting epoch 1 / 5


100%|██████████| 40/40 [00:08<00:00,  4.88it/s, acc=0.151, loss=3.6]


**Saving model**
Starting epoch 2 / 5


100%|██████████| 40/40 [00:08<00:00,  4.92it/s, acc=0.231, loss=3.12]


**Saving model**
Starting epoch 3 / 5


100%|██████████| 40/40 [00:08<00:00,  4.79it/s, acc=0.265, loss=2.92]


**Saving model**
Starting epoch 4 / 5


100%|██████████| 40/40 [00:08<00:00,  4.91it/s, acc=0.312, loss=2.67]


**Saving model**
Starting epoch 5 / 5


100%|██████████| 40/40 [00:08<00:00,  4.86it/s, acc=0.292, loss=2.85]


In [15]:
state = torch.load(f"checkpoints/r164_c10_pretrained.pth")
model.load_state_dict(state['state_dict'],strict=True)
acc, v_loss = test(model, criterion, optimizer, "test")
print(f"Test Accuracy: {acc} | Valid Accuracy: {state['acc']}")

100%|██████████| 79/79 [00:16<00:00,  4.83it/s, acc=0.299, loss=2.68]

Test Accuracy: 0.2994 | Valid Accuracy: 0.3116
